In [1]:
from __future__ import absolute_import, division, print_function
from tensorflow.keras.layers import Lambda, Dense, Input, Layer, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.initializers import Constant
from tensorflow.keras.backend import concatenate
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K


import gc
import argparse


In [2]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.set_visible_devices(gpus[0], 'GPU')

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
# standard numerical library imports
import numpy as np
import math
from math import *
import scipy as sp
import matplotlib.pyplot as plt
import os

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split, to_categorical


In [4]:
# Global plot settings
from matplotlib import rc
import matplotlib.font_manager
rc('font', family='serif')
rc('text', usetex=False)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)


pythia_text = r'$POWHEG \; (hvq) \; pp \to  t\bar{t}$'
def make_legend(ax, title):
    leg = ax.legend(frameon=False)
    leg.set_title(title, prop={'size':20})
    leg.texts[0].set_fontsize(20)
    leg._legend_box.align = "left"
    plt.tight_layout()


import mplhep as hep
plt.style.use(hep.style.CMS)


In [5]:
# define training iter function and helpful setup_nn function

def setup_nn(Phi_sizes = (100,100,128), F_sizes = (100,100,100), input_dim=6, patience = 15, save_label = 'DCTR_training', out_dir = './saved_models'):

    dctr = PFN(input_dim = input_dim,
               Phi_sizes = Phi_sizes, 
               F_sizes   = F_sizes,
               summary   = False)

    os.makedirs(os.path.dirname(f'{out_dir}/{save_label}.h5'), exist_ok=True) # create output dir, if it doesn't exist
    
    checkpoint = keras.callbacks.ModelCheckpoint(f'{out_dir}/{save_label}.h5',
                                                    monitor='val_loss',
                                                    verbose=2,
                                                    save_best_only=True,
                                                    mode='min')
    
    CSVLogger = keras.callbacks.CSVLogger(f'{out_dir}/{save_label}_loss.csv', append=False)
    
    EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  min_delta=0,
                                                  patience=patience,
                                                  verbose=1,
                                                  restore_best_weights=True)
    
    callbacks = [checkpoint, CSVLogger, EarlyStopping]

    return dctr, callbacks


def train_single_iteration(X0, X1, iteration, num_events = int(4e7), batch_size = (5*8192), save_label = 'DCTR_training', out_dir = './saved_models'):
    K.clear_session()
    gc.collect() # collect garbage to free memory
    
    # take random num_evts from each dataset
    rand0 = np.random.choice(len(X0), num_events) # num_evts random indices
    rand1 = np.random.choice(len(X1), num_events)

    local_X0 = X0[rand0]
    local_X1 = X1[rand1]

    # create classifier array Y
    Y0 = np.array([0. for i in range(local_X0.shape[0])])
    Y1 = np.array([1. for i in range(local_X1.shape[0])])

    Y = np.concatenate((Y0, Y1))
    Y = to_categorical(Y, num_classes=2)

    # create training array
    X = []
    X = np.concatenate((local_X0, local_X1))
    
    X_train, X_val, Y_train, Y_val = data_split(X, Y, test=0.25, shuffle=True)

    del rand0, rand1, local_X0, local_X1, X, Y # delete the tmp variables to free memory
    gc.collect() # collect garbage to free memory

    with tf.device('/cpu:0'):
        X_train = tf.convert_to_tensor(X_train)
        X_val   = tf.convert_to_tensor(X_val)
        Y_train = tf.convert_to_tensor(Y_train)
        Y_val   = tf.convert_to_tensor(Y_val)
    
    # ready to start training
    dctr, callbacks = setup_nn(save_label = f'{save_label}_iter_{iteration:02d}', out_dir = out_dir)

    print('Starting training')
    history = dctr.fit(X_train, Y_train,
                       epochs = 1000,
                       batch_size = batch_size,
                       validation_data = (X_val, Y_val),
                       verbose = 1,
                       callbacks = callbacks)
    
    dctr.save(f'{out_dir}/{save_label}_iter_{iteration:02d}.h5')
    
    plt.figure(figsize=(6,5))
    plt.plot(history.history['loss'],     label = 'loss')
    plt.plot(history.history['val_loss'], label = 'val loss')
    plt.legend(loc=0)
    plt.ylabel('loss')
    plt.xlabel('Epochs')
    plt.savefig(f'{out_dir}/{save_label}_iter_{iteration:02d}_history.pdf')

    del history, dctr, callbacks, X_train, X_val, Y_train, Y_val # delete vars to free memory
    K.clear_session()
    gc.collect() # collect garbage to free memory


In [6]:
# reverse engineering normalization function
# pt -> np.log10(pt) ?
# rapidity and phi aren't touched
# mass = mass / max(mass) ?
# pid: top:  6->0.1
#     atop: -6->0.2

# hdamp_val: 1.379

def normalize(X):
    X[:,0,0] = np.log10(X[:,0,0]) # log pt
    X[:,1,0] = np.log10(X[:,1,0])
    
    X[:,0,3] = X[:,0,3]/np.max(X[:,0,3]) # mass
    X[:,1,3] = X[:,1,3]/np.max(X[:,1,3]) 
    
    X[:,0,4] = 0.1 # pid
    X[:,1,4] = 0.2
    
    X[:,0,5] = 1.379 # hdamp
    X[:,1,5] = 1.379

    return X


In [7]:
# convert datasets to normed
'''
data_dir     = '/tf/data/BachelorThesis_Data/Valentinas_Samples'

nominal_list = []
up_list      = []

# X0: down
# X1: nominal

# shape of the datasets in valentinas script was (:, 2, 6)
# shape of the datasets passed to me are         (:, 3, 6)
# so removing 3. entry to be left with only top and anti-top


for i in range(1, 51):
    X0 = []
    X1 = []
    X0_nrm = []
    X1_nrm = []

    
    X0 = np.load(f'{data_dir}/down/Results{i}/2MeventsTrain_0.8738_seed{i}_Base1000_13TeV_P4.npz')['a'][:,:-1,:]    

    X0_nrm = np.array(normalize(X0))

    np.save(f'{data_dir}/down/Results{i}/2MeventsTrain_0.8738_seed{i}_Base1000_13TeV_P4_normed.npy', X0_nrm)

    del X0, X1, X0_nrm, X1_nrm
    gc.collect()
    
    print(f'loaded and normalized nominal and up datasets: \n\
        up to Results{i}')
'''


loaded and normalized nominal and up datasets: 
        up to Results1
loaded and normalized nominal and up datasets: 
        up to Results2
loaded and normalized nominal and up datasets: 
        up to Results3
loaded and normalized nominal and up datasets: 
        up to Results4
loaded and normalized nominal and up datasets: 
        up to Results5
loaded and normalized nominal and up datasets: 
        up to Results6
loaded and normalized nominal and up datasets: 
        up to Results7
loaded and normalized nominal and up datasets: 
        up to Results8
loaded and normalized nominal and up datasets: 
        up to Results9
loaded and normalized nominal and up datasets: 
        up to Results10
loaded and normalized nominal and up datasets: 
        up to Results11
loaded and normalized nominal and up datasets: 
        up to Results12
loaded and normalized nominal and up datasets: 
        up to Results13
loaded and normalized nominal and up datasets: 
        up to Results14
l

In [10]:
# load normed datasets

data_dir     = '/tf/data/BachelorThesis_Data/Valentinas_Samples'

# X0: down
# X1: nominal
X0_nrm = []
X1_nrm = []


for i in range(1, 51):
    X0_nrm.extend(np.load(f'{data_dir}/down/Results{i}/2MeventsTrain_0.8738_seed{i}_Base1000_13TeV_P4_normed.npy'))
    X1_nrm.extend(np.load(f'{data_dir}/nominal/Results{i}/2MeventsTrain_1.379_seed{i}_Base1000_13TeV_P4_normed.npy'))
    if not i%5: 
        print(f'loaded and added normalized nominal and down datasets: \n\
            up to Results{i}')

X0_nrm = np.array(X0_nrm)
X1_nrm = np.array(X1_nrm)

print(f'{X0_nrm.shape}')
print(f'{X1_nrm.shape}')

gc.collect()


loaded and added normalized nominal and down datasets: 
            up to Results5
loaded and added normalized nominal and down datasets: 
            up to Results10
loaded and added normalized nominal and down datasets: 
            up to Results15
loaded and added normalized nominal and down datasets: 
            up to Results20
loaded and added normalized nominal and down datasets: 
            up to Results25
loaded and added normalized nominal and down datasets: 
            up to Results30
loaded and added normalized nominal and down datasets: 
            up to Results35
loaded and added normalized nominal and down datasets: 
            up to Results40
loaded and added normalized nominal and down datasets: 
            up to Results45
loaded and added normalized nominal and down datasets: 
            up to Results50
(98395223, 2, 6)
(99053382, 2, 6)


1845

In [11]:
# cheack datasets

for part in range(2):
    for arg in range(6):
        print(f'{np.mean(X0_nrm[:, part, arg]) = :.3f}')
        print(f'{np.mean(X1_nrm[:, part, arg]) = :.3f}')



np.mean(X0_nrm[:, part, arg]) = 1.995
np.mean(X1_nrm[:, part, arg]) = 1.996
np.mean(X0_nrm[:, part, arg]) = 0.000
np.mean(X1_nrm[:, part, arg]) = 0.000
np.mean(X0_nrm[:, part, arg]) = 0.000
np.mean(X1_nrm[:, part, arg]) = 0.000
np.mean(X0_nrm[:, part, arg]) = 0.705
np.mean(X1_nrm[:, part, arg]) = 0.705
np.mean(X0_nrm[:, part, arg]) = 0.100
np.mean(X1_nrm[:, part, arg]) = 0.100
np.mean(X0_nrm[:, part, arg]) = 1.379
np.mean(X1_nrm[:, part, arg]) = 1.379
np.mean(X0_nrm[:, part, arg]) = 1.996
np.mean(X1_nrm[:, part, arg]) = 1.997
np.mean(X0_nrm[:, part, arg]) = 0.000
np.mean(X1_nrm[:, part, arg]) = 0.000
np.mean(X0_nrm[:, part, arg]) = 0.000
np.mean(X1_nrm[:, part, arg]) = 0.000
np.mean(X0_nrm[:, part, arg]) = 0.705
np.mean(X1_nrm[:, part, arg]) = 0.705
np.mean(X0_nrm[:, part, arg]) = 0.200
np.mean(X1_nrm[:, part, arg]) = 0.200
np.mean(X0_nrm[:, part, arg]) = 1.379
np.mean(X1_nrm[:, part, arg]) = 1.379


In [ ]:
# 50 iterations with 40M random samples per iter

for i in range(1, 51):
    train_single_iteration(X0_nrm, X1_nrm, iteration=i, num_events = int(4e7), save_label = 'DCTR_training_40M_down')


2024-06-19 14:11:54.462111: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5760000000 exceeds 10% of free system memory.


Starting training
Epoch 1/1000
1465/1465 [==============================] - ETA: 0s - loss: 0.6956 - acc: 0.5027
Epoch 1: val_loss improved from inf to 0.69327, saving model to ./saved_models/DCTR_training_40M_down_iter_01.h5
1465/1465 [==============================] - 21s 13ms/step - loss: 0.6956 - acc: 0.5027 - val_loss: 0.6933 - val_acc: 0.5050
Epoch 2/1000
1461/1465 [============================>.] - ETA: 0s - loss: 0.6931 - acc: 0.5049
Epoch 2: val_loss improved from 0.69327 to 0.69318, saving model to ./saved_models/DCTR_training_40M_down_iter_01.h5
1465/1465 [==============================] - 18s 13ms/step - loss: 0.6931 - acc: 0.5049 - val_loss: 0.6932 - val_acc: 0.5057
Epoch 3/1000
1463/1465 [============================>.] - ETA: 0s - loss: 0.6930 - acc: 0.5066
Epoch 3: val_loss improved from 0.69318 to 0.69311, saving model to ./saved_models/DCTR_training_40M_down_iter_01.h5
1465/1465 [==============================] - 18s 13ms/step - loss: 0.6930 - acc: 0.5066 - val_loss: 

2024-06-19 14:25:03.178573: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5760000000 exceeds 10% of free system memory.


Starting training
Epoch 1/1000
1464/1465 [============================>.] - ETA: 0s - loss: 0.6962 - acc: 0.5035
Epoch 1: val_loss improved from inf to 0.69324, saving model to ./saved_models/DCTR_training_40M_down_iter_02.h5
1465/1465 [==============================] - 21s 13ms/step - loss: 0.6962 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.5054
Epoch 2/1000
1465/1465 [==============================] - ETA: 0s - loss: 0.6931 - acc: 0.5053
Epoch 2: val_loss improved from 0.69324 to 0.69314, saving model to ./saved_models/DCTR_training_40M_down_iter_02.h5
1465/1465 [==============================] - 18s 13ms/step - loss: 0.6931 - acc: 0.5053 - val_loss: 0.6931 - val_acc: 0.5062
Epoch 3/1000
1461/1465 [============================>.] - ETA: 0s - loss: 0.6930 - acc: 0.5067
Epoch 3: val_loss improved from 0.69314 to 0.69304, saving model to ./saved_models/DCTR_training_40M_down_iter_02.h5
1465/1465 [==============================] - 19s 13ms/step - loss: 0.6930 - acc: 0.5067 - val_loss: 

2024-06-19 14:41:12.019693: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5760000000 exceeds 10% of free system memory.


Starting training
Epoch 1/1000
1462/1465 [============================>.] - ETA: 0s - loss: 0.6946 - acc: 0.5035
Epoch 1: val_loss improved from inf to 0.69311, saving model to ./saved_models/DCTR_training_40M_down_iter_03.h5
1465/1465 [==============================] - 21s 13ms/step - loss: 0.6946 - acc: 0.5035 - val_loss: 0.6931 - val_acc: 0.5059
Epoch 2/1000
1464/1465 [============================>.] - ETA: 0s - loss: 0.6931 - acc: 0.5058
Epoch 2: val_loss improved from 0.69311 to 0.69297, saving model to ./saved_models/DCTR_training_40M_down_iter_03.h5
1465/1465 [==============================] - 18s 13ms/step - loss: 0.6931 - acc: 0.5058 - val_loss: 0.6930 - val_acc: 0.5079
Epoch 3/1000
1461/1465 [============================>.] - ETA: 0s - loss: 0.6930 - acc: 0.5074
Epoch 3: val_loss did not improve from 0.69297
1465/1465 [==============================] - 18s 13ms/step - loss: 0.6930 - acc: 0.5074 - val_loss: 0.6930 - val_acc: 0.5051
Epoch 4/1000
1461/1465 [=====================

2024-06-19 14:53:55.200899: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5760000000 exceeds 10% of free system memory.


Starting training
Epoch 1/1000
1464/1465 [============================>.] - ETA: 0s - loss: 0.6967 - acc: 0.5031
Epoch 1: val_loss improved from inf to 0.69306, saving model to ./saved_models/DCTR_training_40M_down_iter_04.h5
1465/1465 [==============================] - 21s 13ms/step - loss: 0.6967 - acc: 0.5031 - val_loss: 0.6931 - val_acc: 0.5048
Epoch 2/1000
1464/1465 [============================>.] - ETA: 0s - loss: 0.6932 - acc: 0.5047
Epoch 2: val_loss did not improve from 0.69306
1465/1465 [==============================] - 19s 13ms/step - loss: 0.6932 - acc: 0.5047 - val_loss: 0.6934 - val_acc: 0.5004
Epoch 3/1000
1464/1465 [============================>.] - ETA: 0s - loss: 0.6930 - acc: 0.5062
Epoch 3: val_loss did not improve from 0.69306
1465/1465 [==============================] - 19s 13ms/step - loss: 0.6930 - acc: 0.5062 - val_loss: 0.6932 - val_acc: 0.5058
Epoch 4/1000
1464/1465 [============================>.] - ETA: 0s - loss: 0.6930 - acc: 0.5072
Epoch 4: val_loss im